In [10]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col}

In [2]:
val spark = SparkSession.builder().appName("User Id Rating History").master("local[*]").getOrCreate()

spark = org.apache.spark.sql.SparkSession@2c0908a5


org.apache.spark.sql.SparkSession@2c0908a5

In [3]:
val sprk_ctx = spark.sparkContext

sprk_ctx = org.apache.spark.SparkContext@22418b51


org.apache.spark.SparkContext@22418b51

#### Read csv from GCS

In [7]:
val rating_gcs_path = "gs://artifacts_spark/rating.csv"
val movie_gcs_path = "gs://artifacts_spark/movie.csv"

val rating_df = spark.read.format("csv").option("header",true).option("inferSchema", true).load(rating_gcs_path)
val movie_df = spark.read.format("csv").option("header", true).option("inferSchema", true).load(movie_gcs_path)

rating_gcs_path = gs://artifacts_spark/rating.csv
movie_gcs_path = gs://artifacts_spark/movie.csv
rating_df = [userId: int, movieId: int ... 2 more fields]
movie_df = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [8]:
val joined_df = movie_df.join(rating_df, "movieId")

joined_df = [movieId: int, title: string ... 4 more fields]


[movieId: int, title: string ... 4 more fields]

In [9]:
joined_df.show(5)

+-------+--------------------+--------------------+------+------+-------------------+
|movieId|               title|              genres|userId|rating|          timestamp|
+-------+--------------------+--------------------+------+------+-------------------+
|      2|      Jumanji (1995)|Adventure|Childre...|     1|   3.5|2005-04-02 23:53:47|
|     29|City of Lost Chil...|Adventure|Drama|F...|     1|   3.5|2005-04-02 23:31:16|
|     32|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|     1|   3.5|2005-04-02 23:33:39|
|     47|Seven (a.k.a. Se7...|    Mystery|Thriller|     1|   3.5|2005-04-02 23:32:07|
|     50|Usual Suspects, T...|Crime|Mystery|Thr...|     1|   3.5|2005-04-02 23:29:40|
+-------+--------------------+--------------------+------+------+-------------------+
only showing top 5 rows



#### Filter out the null columns

In [12]:
val filteredData = joined_df.filter(col("movieId").isNotNull && col("title").isNotNull && col("genres").isNotNull)

filteredData = [movieId: int, title: string ... 4 more fields]


[movieId: int, title: string ... 4 more fields]

In [13]:
val limitedData = filteredData.limit(1000)

limitedData = [movieId: int, title: string ... 4 more fields]


[movieId: int, title: string ... 4 more fields]

In [17]:
val output_path = "hdfs:///user/mano/2_caseStudy/"

limitedData.write.partitionBy("userId").format("parquet").save(output_path)

output_path = hdfs:///user/mano/2_caseStudy/


hdfs:///user/mano/2_caseStudy/

In [18]:
spark.stop()